In [1]:
#import libraries
import numpy as np
import pandas as pd

# Importing Data

In [2]:
steam_data = pd.read_csv('AllSteamData.csv', header = None)
steam_data.head()

C:\Users\choww\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5
0,Name,Month,Avg. Players,Gain,% Gain,Peak Players
1,Counter-Strike,Last 30 Days,8488.74,27.6,0.33%,15065
2,Counter-Strike,Sep-21,8461.12,-390.07,-4.41%,14559
3,Counter-Strike,Aug-21,8851.19,-286.57,-3.14%,14064
4,Counter-Strike,Jul-21,9137.76,-359.69,-3.79%,14972


In [3]:
steam_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878176 entries, 0 to 878175
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       878176 non-null  object
 1   1       878176 non-null  object
 2   2       878176 non-null  object
 3   3       867342 non-null  object
 4   4       867342 non-null  object
 5   5       878176 non-null  object
dtypes: object(6)
memory usage: 40.2+ MB


In [4]:
steam_data

,0,1,2,3,4,5
0,Name,Month,Avg. Players,Gain,% Gain,Peak Players
1,Counter-Strike,Last 30 Days,8488.74,27.6,0.33%,15065
2,Counter-Strike,Sep-21,8461.12,-390.07,-4.41%,14559
3,Counter-Strike,Aug-21,8851.19,-286.57,-3.14%,14064
4,Counter-Strike,Jul-21,9137.76,-359.69,-3.79%,14972
...,...,...,...,...,...,...
878171,Ultra Age Demo,Last 30 Days,5.45,NaN,NaN,35
878172,Grim Tales: Crimson Hollow Collector's Edition,Last 30 Days,2.4,NaN,NaN,9
878173,Everyday Life in Hospital VR,Last 30 Days,0.0,NaN,NaN,0
878174,Martha Is Dead Demo,Last 30 Days,9.03,NaN,NaN,41
